In [2]:
import gym
import random
env = gym.make('SpaceInvaders-v0')
height, width,channels =env.observation_space.shape 
actions = env.action_space.n

C:\Users\An\anaconda3\envs\tf4gym\lib\site-packages\ale_py\roms\utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [3]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [4]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [5]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [16]:
del model

In [17]:
model = build_model(height, width, channels, actions)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_8 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_9 (Dense)              (None, 6)                

In [18]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import BoltzmannQPolicy

In [1]:
from tensorflow import keras 
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=.7,
    decay_steps=10000,
    decay_rate=0.01)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)

In [2]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=100 , gamma=.8
                  )
    return dqn

In [3]:
dqn = build_agent(model, actions)
dqn.compile(Adam(epsilon=1.))

NameError: name 'model' is not defined

In [26]:
dqn.fit(env, nb_steps=5000, visualize=False, verbose=2)

Training for 5000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
  926/5000: episode: 1, duration: 756.923s, episode steps: 926, steps per second:   1, episode reward: 140.000, mean reward:  0.151 [ 0.000, 20.000], mean action: 2.529 [0.000, 5.000],  loss: 0.564472, mean_q: -1.128461
 1557/5000: episode: 2, duration: 611.165s, episode steps: 631, steps per second:   1, episode reward: 115.000, mean reward:  0.182 [ 0.000, 25.000], mean action: 2.585 [0.000, 5.000],  loss: 0.581945, mean_q: -1.159402
 2183/5000: episode: 3, duration: 596.627s, episode steps: 626, steps per second:   1, episode reward: 90.000, mean reward:  0.144 [ 0.000, 25.000], mean action: 2.436 [0.000, 5.000],  loss: 0.419707, mean_q: -1.376690
 3081/5000: episode: 4, duration: 847.573s, episode steps: 898, steps per second:   1, episode reward: 325.000, mean reward:  0.362 [ 0.000, 200.000], mean action: 2.575 [0.000, 5.000],  loss: 6.7

In [27]:
dqn.save_weights('d:/TensorflowModule/SavedWeights/base_line_nonGreedy/dqn_weights.h5f')

In [23]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Episode 1: reward: 80.000, steps: 975
Episode 2: reward: 40.000, steps: 671
Episode 3: reward: 140.000, steps: 972
Episode 4: reward: 125.000, steps: 1123
Episode 5: reward: 125.000, steps: 987
Episode 6: reward: 75.000, steps: 716
Episode 7: reward: 300.000, steps: 822
Episode 8: reward: 85.000, steps: 796
Episode 9: reward: 55.000, steps: 557
Episode 10: reward: 80.000, steps: 907
Episode 11: reward: 220.000, steps: 1091
Episode 12: reward: 75.000, steps: 688
Episode 13: reward: 55.000, steps: 716
Episode 14: reward: 60.000, steps: 667
Episode 15: reward: 15.000, steps: 633
Episode 16: reward: 55.000, steps: 695
Episode 17: reward: 60.000, steps: 706
Episode 18: reward: 50.000, steps: 487
Episode 19: reward: 115.000, steps: 1051
Episode 20: reward: 105.000, steps: 1009
Episode 21: reward: 15.000, steps: 536
Episode 22: reward: 55.000, steps

In [ ]:
del model, dqn

In [22]:
dqn.load_weights('d:/TensorflowModule/SavedWeights/base_line_nonGreedy/dqn_weights.h5f')